In [2]:
import pandas as pd
from textblob import TextBlob
import nest_asyncio
import requests
from bs4 import BeautifulSoup
import aiohttp
import asyncio

In [8]:
# Permitir la ejecución de eventos en el mismo hilo
nest_asyncio.apply()

# Lexicon de palabras
lexicon = ["migrantes"]

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status != 200:
            print(f"Error: {response.status} - {url}")
        return await response.text()

async def get_tweets_for_keyword(session, keyword):
    url = f"https://nitter.poast.org/search?f=tweets&q=migrantes&since=&until=&near="
    html = await fetch(session, url)
    soup = BeautifulSoup(html, 'html.parser')
    tweets = []
    for tweet in soup.find_all('div', {'class': 'timeline-item'}):
        content_div = tweet.find('div', {'class': 'tweet-content'})
        if content_div:
            content = content_div.text.strip()
        else:
            content = "No content found"
        date_span = tweet.find('span', {'class': 'tweet-date'})
        if date_span:
            date = date_span.text.strip()
        else:
            date = "No date found"
        tweets.append({'keyword': keyword, 'content': content, 'date': date})
    return tweets

async def main():
    async with aiohttp.ClientSession() as session:
        all_tweets = []
        for keyword in lexicon:
            print(f"Fetching tweets for keyword: {keyword}")
            tweets = await get_tweets_for_keyword(session, keyword)
            all_tweets.extend(tweets)
        return all_tweets

# Ejecutar la recolección de tweets
loop = asyncio.get_event_loop()
tweets_data = loop.run_until_complete(main())

# Convertir a DataFrame de pandas
tweets_df = pd.DataFrame(tweets_data)

# Guardar los resultados en un archivo CSV
tweets_df.to_csv('tweets_recolectados_nitter.csv', index=False)

# Mostrar los primeros 5 tweets para ver el resultado
print(tweets_df.head())

Fetching tweets for keyword: migrantes
Error: 403 - https://nitter.poast.org/search?f=tweets&q=migrantes&since=&until=&near=
Empty DataFrame
Columns: []
Index: []


In [4]:
display(tweets_data)

[]

In [5]:
display(tweets_df)

""
